In [39]:
import pandas as pd
from pathlib import Path

parent_dir = Path('__file__').parent.absolute()
labels_dir = parent_dir / "metadata"

train_df = pd.read_csv(labels_dir / "FungiCLEF2023_train_metadata_PRODUCTION.csv")

In [40]:
train_df.head()

,observationID,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,phylum,...,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path,class_id,MetaSubstrate,poisonous
0,2238546328,2018.0,4.0,16.0,DK,Ulvedal Plantage,30872.0,Ramalina farinacea (L.) Ach.,Fungi,Ascomycota,...,bark of living trees,Ulfva Melchior Hvidegaard,56.299706,9.258110,50.0,Mixed woodland (with coniferous and deciduous ...,2238546328-30620.JPG,1273,wood,0
1,2558871973,2020.0,1.0,3.0,DK,Slotshegn,15256.0,Hysterium acuminatum Fr.,Fungi,Ascomycota,...,dead wood (including bark),Ole Martin,55.861899,11.975973,50.0,Deciduous woodland,2558871973-53941.JPG,708,wood,0
2,2238503501,2017.0,8.0,22.0,DK,Petersborg Strandenge,61200.0,Gliophorus perplexus (A.H.Sm. & Hesler) Kovalenko,Fungi,Basidiomycota,...,soil,Per Taudal Poulsen,56.975158,9.285525,75.0,natural grassland,2238503501-245559.JPG,535,jord,0
3,2446759075,2019.0,10.0,26.0,DK,Klintebjerg,30530.0,Lecidella scabra (Taylor) Hertel & Leuckert,Fungi,Ascomycota,...,stone,Susanne Rabenborg,55.960242,11.583103,15.0,gravel or clay pit,2446759075-197643.JPG,832,stone,0
4,2238472345,2016.0,8.0,21.0,DK,Blåbjerg,63728.0,"Russula fragilis Fr., 1838",Fungi,Basidiomycota,...,soil,Tom Smidth,55.742985,8.250188,50.0,Mixed woodland (with coniferous and deciduous ...,2238472345-167057.JPG,1338,jord,0


In [41]:
train_df["image_path"].is_unique 

True

In [42]:
mask = train_df["observationID"].duplicated()
sum(mask)

118768

In [43]:
# there are duplicate observationIDs because each observation can have more than one associated image

In [44]:
# train_df = train_df.drop_duplicates("observationID", keep="first")

In [45]:
train_df["image_path"].is_unique 

True

In [46]:
train_df["class_id"].nunique()

1604

In [47]:
def load_clean_df(df_path: Path):
    df = pd.read_csv(df_path)
    return df

In [48]:
def load_clean_get_nunique(df_path: Path):
    df = pd.read_csv(df_path)
    return df["class_id"].nunique()

In [49]:
val_df_path = labels_dir / "FungiCLEF2023_val_metadata_PRODUCTION.csv"
load_clean_get_nunique(val_df_path)

1085

In [50]:
val_df = load_clean_df(val_df_path)

In [51]:
val_df["image_path"].is_unique

True

In [52]:
val_df["class_id"].value_counts()

-1       21954
 937       458
 1486      457
 492       355
 1437      307
         ...  
 549         1
 1443        1
 1475        1
 204         1
 314         1
Name: class_id, Length: 1085, dtype: int64

In [62]:
val_df["class_id"].value_counts()[1:21]

937     458
1486    457
492     355
1437    307
88      293
99      289
1182    270
515     265
1491    264
105     250
1482    247
133     240
973     239
1483    233
1184    227
1460    225
931     217
387     217
170     203
744     202
Name: class_id, dtype: int64

In [53]:
train_df["class_id"].value_counts()

1486    1913
492     1743
687     1636
1186    1365
489     1317
        ... 
1407      31
833       31
1344      31
945       31
1091      31
Name: class_id, Length: 1604, dtype: int64

In [63]:
train_df["class_id"].value_counts()[:20]

1486    1913
492     1743
687     1636
1186    1365
489     1317
99      1301
88      1270
249     1256
1437    1225
973     1221
1184    1167
648     1117
225     1115
515     1107
387     1065
1182    1034
1491    1033
553      989
368      988
931      965
Name: class_id, dtype: int64

In [68]:
n_classes = 20
len(set(train_df["class_id"].value_counts()[:n_classes].index)
.intersection(val_df["class_id"].value_counts()[1:n_classes+1].index))

12

In [67]:
len(set(train_df["class_id"].value_counts()[:n_classes].index)
    .symmetric_difference(val_df["class_id"].value_counts()[1:n_classes+1].index))

16

# are the training and validation classes the same?
- no.
- there is an "unknown" class in the validation set (-1) that is not in the training set
- there are many classes in the trianing set not in the validation set

In [54]:
train_classes = set(train_df["class_id"].unique())
val_classes = set(val_df["class_id"].unique())

In [55]:
len(train_classes - val_classes)

520

In [56]:
val_classes - train_classes

{-1}